In [48]:
!git clone https://github.com/mathav95raj/NMF-for-event-detection-in-overlapping-signals.git

fatal: destination path 'NMF-for-event-detection-in-overlapping-signals' already exists and is not an empty directory.


In [49]:
import matplotlib.pyplot as plt
import os
import librosa.core as lc
import numpy as np
from sklearn import preprocessing
import pandas as pd

In [50]:
def loadfilenames(path):
    textfiles = []
    wavfiles = []
    for file in os.listdir(path):
      if file.endswith( ".txt" ):
          textfiles.append(path + os.sep + file)
      if file.endswith( ".wav" ):
          wavfiles.append(path+ os.sep+ file)
    textfiles = np.array(textfiles).reshape(-1,1)
    wavfiles = np.array(wavfiles).reshape(-1,1)
    return textfiles, wavfiles

In [51]:
def loaddata(wavfile, textfile):
    data, sr = lc.load(wavfile, mono=True)
    df = pd.read_csv(textfile, delimiter = '\t', header = None)
    label = df[2].tolist()
    df = df.drop(columns = 2)
    return data, sr, df, label

In [52]:
def stftmatrixgen(data):
    stftMat = np.abs(lc.stft(data, n_fft = _n_fft,  hop_length= hop, center=False))
    return stftMat

In [53]:

def litodic(label):
    arr = np.unique(label)
    dic = {i:str(arr[i]) for i in range(len(arr))}
    return dic

In [54]:
def onehot_label(df, label, dic):
    df_temp = pd.DataFrame(columns = list(dic.values()))
    df = pd.concat([df,pd.DataFrame(columns = list(dic.values()))])
    df[df_temp.columns] = 0
    for i in range(len(label)):
        df[label[i]][i] = 1
    return df

In [55]:
def insertrow(row_no, df, row_val):
    df1 = df[0:row_no]
    df2 = df[row_no:]
    df1.loc[row_no] = row_val
    df_result = pd.concat([df1, df2])
    df_result.index = [*range(df_result.shape[0])]
    return df_result 

In [56]:
def findslots(df, f_len):
    for i in range(len(df) - 1):
        if df[1][i] > df[0][i+1]:
            row_val = [df[0][i+1], df[1][i]]
            row_val.extend(np.array(df.iloc[i,2:]) + np.array(df.iloc[i+1,2:]))
            df[1][i] = df[0][i+1] - f_len
            df = insertrow(i+1, df, row_val)
            df[0][i+2] = df[1][i+1] + f_len    
    slot = np.round((1 / f_len) * (df[[0,1]]))
    return df, slot

In [57]:
def Nd_slotgen(N_slot):
    Nd_slot = []
    for i in range(D):
        Nd_slot.append((N_slot.loc[ np.where(K_slot.iloc[:,i] == 1) ]).to_numpy())
    return Nd_slot

In [58]:
def maskmatrix(A):
    for i in range(D):
        for j in range(Nd_slot[i].shape[0]):
            A [i*kd : (i+1)*kd , int(Nd_slot[i][j][0]) : int(Nd_slot[i][j][1] + 1) ] = 0
    return A

In [59]:
def initialize(M, N, K):
    W = np.random.rand(M,K)
    H = np.random.rand(K,N)
    E = np.ones(M*M).reshape(M,M)
    A = np.ones(K*N).reshape(K, N).astype(int) 
    return W, H, E, A

In [60]:
def nmfitrtrain(W, H, E, A, X):
    for i in range(iterations):
        W = preprocessing.normalize(W.T,norm='l1')
        W = W.T
        H = H*(np.dot(W.T,X)/(np.dot(np.dot(W.T,W),H)+lambd*A))
        W = W*((np.dot(X,H.T) + np.dot(E,((np.dot(np.dot(W,H),H.T))*W)))/((np.dot(np.dot(W,H),H.T)) + (np.dot(np.dot(E,X),H.T))*W))
        error = np.linalg.norm(X - np.dot(W,H))
        print(error)
    return W, H, E, A

In [61]:
def nmfitrtest(W, H, X):
    for i in range(iterations):
        H = H*(np.dot(W.T,X)/(np.dot(np.dot(W.T,W),H)))
        error = np.linalg.norm(X - np.dot(W,H))
        print('testing', error)
    return H 

In [62]:
_n_fft = 1024
hop = 160
trainpath = 'NMF-for-event-detection-in-overlapping-signals/train'
testpath = 'NMF-for-event-detection-in-overlapping-signals/test'
textfiles, wavfiles = loadfilenames(trainpath)    
files = np.concatenate((wavfiles, textfiles), axis = 1)

c = 0
for file in files:
    data, sr, df, label = loaddata(file[0], file[1])
    dic = litodic(label)
    df = onehot_label(df, label, dic)
    stftMat = stftmatrixgen(data)
    X = stftMat
    f_len = (len(data)/sr) / stftMat.shape[1]
    df, N_slot = findslots(df, f_len)
    K_slot = df.iloc[:,2:]
    M = stftMat.shape[0]
    N = stftMat.shape[1]
    D = len(dic)
    kd = 10
    K = D * kd
    Nd_slot = Nd_slotgen(N_slot)
    if c == 0:
        W, H, E, A = initialize(M, N, K)
        c = 1
    A = maskmatrix(A)
    iterations = 5
    lambd = 10
    W, H, E, A = nmfitrtrain(W, H, E, A, X)  
    
textfiles, wavfiles = loadfilenames(testpath)    
files = np.concatenate((wavfiles, textfiles), axis = 1)
result = []
ground = []
for file in files:
    data, sr, df, label = loaddata(file[0], file[1])
    dic = litodic(label)
    df = onehot_label(df, label, dic)
    R = df.iloc[:,2:]
    ground.append(R)
    stftMat = stftmatrixgen(data)
    X = stftMat
    f_len = (len(data)/sr) / stftMat.shape[1]
    df, N_slot = findslots(df, f_len)
    K_slot = df.iloc[:,2:]
    M = stftMat.shape[0]
    N = stftMat.shape[1]
    D = len(dic)
    kd = 10
    K = D * kd
    Nd_slot = Nd_slotgen(N_slot)
    iterations = 5
    lambd = 10
    H = np.random.rand(K,N)
    H = nmfitrtest(W, H, X)  
    S = np.zeros((D, len(N_slot)))
    for i in range(D):
        for j in range(N_slot.shape[0]):
            S[i][j] = np.sum(H[i*kd : (i+1)*kd , int(N_slot.iloc[j,0]) : int(N_slot.iloc[j,1] + 1)])            
    result.append(S)
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


161.42837160321304
149.6007022416544
144.7546861364205
145.80693956285148
144.8433571869536


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


94.21293118128158
90.44869428126603
89.9044437813696
89.66808179101461
89.30079991810594


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


...

testing 178.36217818620804
testing 177.13557704547924
testing 175.8253553699444
testing 174.42248709195556
testing 172.9653291004429


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


testing 168.0924187980996
testing 166.87206820396446
testing 165.54767419496162
testing 164.10760182855904
testing 162.60267055663272


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


testing 101.22698635598783
testing 100.36072213984836
testing 99.49446407647251
testing 98.6244592572595
testing 97.75675679365924
